# 利用 Amazon SageMaker 托管的推理环境，部署人脸生成模型

### Environment setup
Upgrade packages

In [ ]:
!pip install --upgrade pip -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install --upgrade sagemaker awscli boto3 pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

Following commands for ```SageMaker Studio``` only

In [ ]:
!pip uninstall -y tqdm

import os
os.chdir('/root/ml-on-aws/byos-pytorch-gan')

!pwd

### Download pretrained PGAN-CelebAHQ-512 model

We'll download [Progressive GAN](https://arxiv.org/abs/1710.10196) pre-trained model [PGAN-CelebAHQ-512](https://pytorch.org/hub/facebookresearch_pytorch-gan-zoo_pgan/) from Torch Hub, which is trained on high-quality celebrity faces "celebAHQ" dataset, then create a model artifact `model.tar.gz` and upload it to S3:

In [ ]:
%%time
import torch

use_gpu = False
model_algorithm = 'PGAN'

# this model outputs 512 x 512 pixel images
model_pretrained_name = 'celebAHQ-512' # 'celebAHQ-256'

pgan = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
                       model_algorithm, model_name=model_pretrained_name,
                       pretrained=True, useGPU=use_gpu)

# generator network
netG = pgan.netG

num_latent_dim = 512
print(netG.getOutputSize())
netG

### Test generator model with random noises.

In [ ]:
import torch
from networks.PGAN.model_tools import generate_noises, show_multiple_pictures

noises = generate_noises(4, num_latent_dim)

with torch.no_grad():
    images = netG(noises)

show_multiple_pictures(images)
# print(netG)

### Save PGAN model to a `.pth` file, and create a TorchServe archive from it.


In [ ]:
import time

model_name = f'model-{model_algorithm}-{model_pretrained_name}-' \
                    + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

model_folder = f'./tmp/{model_algorithm}'
!mkdir -p {model_folder}

input_shape = [1, num_latent_dim]
# data_shape = '{"input0":[1, 512]}'

traced_model = torch.jit.trace(netG.float().eval(), torch.zeros(input_shape).float())
# scripted_model = torch.jit.script(model)
traced_model.save(f"{model_folder}/{model_name}.pth")

### Create a compressed tar.gz file from the model.pth file.
Amazon SageMaker expects that model and related resources are in a tar.gz file, please check manual [SageMaker SDK (using_pytorch)](https://sagemaker.readthedocs.io/en/stable/using_pytorch.html#create-the-directory-structure-for-your-model-files) for details.
<code>
model.tar.gz/
  |- model.pth
  |- code/
    |- inference.py
    |- requirements.txt  # only for versions 1.3.1 and higher
</code>

In [ ]:
!rm -rf ./model/*

!cp -af {model_folder}/{model_name}.pth ./model/model.pth
!cp -af ./networks/PGAN ./model/code
!rm -rf ./model/code/__pycache__

In [ ]:
%%time
import tarfile

with tarfile.open(f"{model_folder}/{model_name}.tar.gz", 'w:gz') as f:
    f.add(f"./model/model.pth", arcname="model.pth")
    f.add(f"./model/code", arcname="code")

### Get current SageMaker session, default S3 bucket, and get a role with SageMaker access.

In [ ]:
import sagemaker
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()

sess = sagemaker.Session()
bucket = sess.default_bucket()

### Upload the generated model archive file to Amazon S3
Uploads the model to your default Amazon SageMaker S3 bucket under the models directory

In [ ]:
from sagemaker.s3 import S3Uploader as s3up

s3_model_location = s3up.upload(f"{model_folder}/{model_name}.tar.gz", f"s3://{bucket}/artifacts/models")

### Define a class GanPredictor, with serializer and deserializer for x-npy format

In [ ]:
from sagemaker.predictor import RealTimePredictor
from networks.PGAN.serde import serialize, deserialize

content_type = 'application/x-npy'

class GanPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session,
                         serializer=serialize, deserializer=deserialize,
                         content_type=content_type, accept=content_type)

### Create a PyTorch model from model archive

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

sm_model = PyTorchModel(model_data=s3_model_location,
                     role=role,
                     predictor_cls=GanPredictor,
                     name=model_name,
                     entry_point='inference.py',
                     py_version="py3",
                     framework_version="1.5",
                     sagemaker_session=sess)

### Deploy model

In [ ]:
%%time
import time

endpoint_name = f'endpoint-{model_algorithm}-{model_pretrained_name}-' \
                    + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

predictor = sm_model.deploy(endpoint_name=endpoint_name,
                         instance_type='ml.c5.xlarge',
                         initial_instance_count=1)
                            

### Invoke the endpoint

Let's generate some face images.

In [ ]:
%%time
from networks.PGAN.model_tools import generate_noises, show_multiple_pictures

images = []
for i in range(0, 2):
    noises = generate_noises(1, num_latent_dim)
    output = predictor.predict(noises.numpy())
    images.extend(output)

show_multiple_pictures(images)

### Delete model and endpoint resources

In [ ]:
sm_model.delete_model()
predictor.delete_endpoint(delete_endpoint_config=True)

### Debug

In [ ]:
!pip install sagemaker_inference -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
%cd ./model/code
import json
from inference import context
from model_tools import generate_noises, show_multiple_pictures
from serde import serialize, deserialize
import handler
%cd ../..

ctxt = context("./model")

noises = generate_noises(4, dim=512)
x = serialize(noises.numpy())

output = handler.handle([{'data':x}], ctxt)

show_multiple_pictures(output[0])

In [ ]:
%cd ./model/code
import json
import inference
from model_tools import generate_noises, show_multiple_pictures
from serde import serialize, deserialize

model = inference.model_fn("..")
%cd ../..

content_type = 'application/x-npy'

noises = generate_noises(4, dim=512)
noises_serialized = serialize(noises.numpy())

input_data = inference.input_fn(noises_serialized, content_type)

prediction = inference.predict_fn(input_data, model)
images_serialized = inference.output_fn(prediction, content_type)

show_multiple_pictures(deserialize(images_serialized, content_type))